In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Sentiment analysis/final_dataset.csv', usecols = ['reviews.rating', 'reviews.text'],encoding= 'unicode_escape')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head(10)

,reviews.rating,reviews.text
0,3,I thought it would be as big as small paper bu...
1,5,This kindle is light and easy to use especiall...
2,4,Didnt know how much i'd use a kindle so went f...
3,5,I am 100 happy with my purchase. I caught it o...
4,5,Solid entry level Kindle. Great for kids. Gift...
5,5,This make an excellent ebook reader. Don't exp...
6,5,"I ordered this for my daughter, as I have the ..."
7,4,I bought my Kindle about 2 months ago and the ...
8,5,"amazon kindle is always the best ebook, upgrad..."
9,5,"It's beyond my expectation, and it can even sh..."


In [ ]:
df.sample(5)

,reviews.rating,reviews.text
7405,5,Great price and 2 day shipping.
52104,I love reading books this way. This is my 3rd ...,NaN
10032,5,They worked great and last ad long as or longe...
30094,5,This kindle is light and easy to use especiall...
40236,My son loves this thing hasn't put it down yet...,NaN


In [ ]:
df['reviews.rating'].value_counts()

5                                                                                                                                                                                                                                                                                                        23023
4                                                                                                                                                                                                                                                                                                         6696
3                                                                                                                                                                                                                                                                                                         1376
1                                                                                          

In [ ]:
! pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-mqbo5bmk
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-mqbo5bmk
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11754 sha256=0cbd0183bd3b2475bfcdd43808e46013ccae7166eacbfde75197d5d1c41ea773
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mcfert_/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps
import re

In [ ]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [ ]:
df['reviews.text'] = df['reviews.text'].apply(lambda x: get_clean(x))

In [ ]:
 df.head()

,reviews.rating,reviews.text
0,3,i thought it would be as big as small paper bu...
1,5,this kindle is light and easy to use especiall...
2,4,didnt know how much i would use a kindle so we...
3,5,i am 100 happy with my purchase i caught it on...
4,5,solid entry level kindle great for kids gifted...


**SVM**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [ ]:
tfidf = TfidfVectorizer(max_features=40000, ngram_range=(1,5), analyzer='char')

In [ ]:
y = tfidf.fit_transform(df['reviews.text'])
z = df['reviews.rating']

In [ ]:
y.shape, z.shape

((67992, 40000), (67992,))

In [ ]:
y_train, y_test, z_train, z_test = train_test_split(y, z, test_size = 0.2, random_state = 0)

In [ ]:
y_train.shape

(54393, 40000)

In [ ]:
clf = LinearSVC(C=10, class_weight='balanced')
clf.fit(y_train, z_train)

ValueError: ignored

In [ ]:
SVM_pred = clf.predict(y_test)

In [ ]:
print(classification_report(z_test, SVM_pred))

In [ ]:
x = 'this product is really bad. I do not like it'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

In [ ]:
x = 'this product is really good. I liked it so much'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = SVM_pred
label=['1', '2','3','4', '5']
Ytrue = z_test 
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for SVM")
ax = sns.heatmap(cm, cmap="mako", fmt=".01f",annot_kws={'size':12}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

**Decision Tree Classifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
dt = DecisionTreeClassifier(max_depth=6, random_state=1)
# Predict test set labels
dt.fit(y_train, z_train)
DT_pred = dt.predict(y_test)

# Compute test set accuracy
acc = accuracy_score(z_test, DT_pred)
print("Test set accuracy: {:.2f}".format(acc))

In [ ]:
x = 'this product is really bad. I do not like it'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

In [ ]:
x = 'this product is really good. I liked it so much'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = DT_pred
label=['1', '2','3','4', '5']
Ytrue = z_test 
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for Decision Tree")
ax = sns.heatmap(cm, cmap="mako", fmt=".01f",annot_kws={'size':12}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

**Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB()
clf_NB.fit(y_train, z_train)

In [ ]:
NB_pred = clf_NB.predict(y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(z_test, NB_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(z_test, z_pred))
print(classification_report(z_test, NB_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = NB_pred
label=['1', '2','3','4', '5']
Ytrue = z_test 
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for Naive Bayes")
ax = sns.heatmap(cm, cmap="mako", fmt=".01f",annot_kws={'size':12}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

**K-Nearest Neighbour**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(y_train, z_train)


In [ ]:
KNN_pred = classifier.predict(y_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(z_test, z_pred))
print(classification_report(z_test, KNN_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = KNN_pred
label=['1', '2','3','4', '5']
Ytrue = z_test 
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for K-Nearest Neighbour")
ax = sns.heatmap(cm, cmap="mako", fmt=".01f",annot_kws={'size':12}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

**Gradiant Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import joblib
from  sklearn.utils import parallel_backend 
gboost = GradientBoostingClassifier(learning_rate=.1, n_estimators=500,max_depth=5 )

In [ ]:
gboost.fit(y_train, z_train)
gboost_pred = gboost.predict(z_test)

In [ ]:
from sklearn.metrics import accuracy_score
gboost_acc=accuracy_score(z_test, z_pred)
print(gboost_acc)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = gboost_pred
label=['1', '2','3','4', '5']
Ytrue = z_test 
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for Gradient Boosting")
ax = sns.heatmap(cm, cmap="mako", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

**XG BOOST**

In [ ]:
import xgboost as xgb
xgbd = xgb.XGBClassifier(max_depth=10, n_estimators=5000 , learning_rate=.1, n_jobs=-1)

In [ ]:
xgbd.fit(y_train, z_train)
xgb_pred = xgbd.predict(y_test)
xgb_acc=accuracy_score(z_test, xgb_pred)
print(xgb_acc)

In [ ]:
print(classification_report(z_test, xgb_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = xgb_pred
label=['1', '2','3','4', '5']
Ytrue = y_test 
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for XGBoost")
ax = sns.heatmap(cm, cmap="mako", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

**Neural Network**

In [ ]:
from sklearn.neural_network import MLPClassifier
nnc = MLPClassifier(hidden_layer_sizes=(9),activation="relu",max_iter=1000)
nnc.fit(X_train, y_train)
nnc_predictions =nnc.predict(X_test)
nnc_acc=accuracy_score(y_test, nnc_predictions)
print(nnc_acc)

In [ ]:
print(classification_report(y_test, nnc_predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
Ypred = nnc_predictions
label=['1', '2','3','4', '5','6','7','8']
Ytrue = y_test
cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix for Neural Network")
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
randomf = RandomForestClassifier(n_jobs=-1)
dtree = DecisionTreeClassifier( criterion='entropy')
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(randomf, n_estimators=500000, learning_rate=.5)
ab.fit(y_train, z_train)
ab_pred = ab.predict(y_test)
ab_acc=accuracy_score(z_test, ab_pred)
print(ab_acc)